## **READ BEFORE PROCEEDING**

- Inside the `input` and `output` directories, make sure you create a directory for each phenotype you plan on executing the ChatGPT code for
- You will need to upload the files generated by ATLAS into the appropriate directory and name them according to the filenames specified in the first code block for each phenotype of interest
- Create an OpenAI account, generate an OpenAI API key (you may need to deposit a small API balance to use the key), and paste the API key in the .env file located so that ChatGPT responses can be generated

In [1]:
import sys
!{sys.executable} -m pip install -q openai
!{sys.executable} -m pip install -q python-dotenv
!{sys.executable} -m pip install -q pandas


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
!{sys.executable} -m pip install --upgrade openai

  Obtaining dependency information for openai from https://files.pythonhosted.org/packages/11/e0/4ddc922d454cc96dcdbc1c0b64fb886b8633fbe470b6e4c749dce06ab6fa/openai-1.32.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 8.6 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.31.2
    Uninstalling openai-1.31.2:
      Successfully uninstalled openai-1.31.2

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [1]:
#Import necessary packages
from openai import OpenAI
import pandas as pd
import re

In [2]:
#Load API key from ENV file and get properties of interest
OPENAI_API_KEY = "ENTER API KEY HERE"
#Instantiate OpenAI client
client = OpenAI(api_key = OPENAI_API_KEY)

In [3]:
def generate_prompt(disease, diseaseDescription, conceptName, domainDefinition, domain = "condition"):
    prompt = f"""You are a helpful medical expert. Your task is to assess whether an inputted {domain} is specific to an inputted malady. Specific means that if you have the {domain}, then you definitely have the malady, but if you have the malady, you may or may not have the {domain}.

A {domain} should be considered {domainDefinition}. 

A description of the malady is provided to assist you, but please use your extensive medical knowledge in addition to this description when performing the task at hand. Provide a “yes” or “no” answer, and explain your rationale for the answer you provide.

Here is the malady:
{disease}

Here is a brief description of the malady:
{diseaseDescription}

Here is the condition:
{conceptName}
"""
    return prompt

In [4]:
def generate_gpt_response(content:str, print_output=False)->str:
    modelName = "gpt-3.5-turbo"
    completions = client.chat.completions.create(#A method that allows you to generate text-based chatbot responses using a pre-trained GPT language model.
        model = modelName, 
        temperature = 0, #controls the level of randomness or creativity in the generated text; . A higher temperature value will result in a more diverse and creative output, as it increases the probability of sampling lower probability tokens. 
#         max_tokens = 2000, #controls the maximum number of tokens (words or subwords) in the generated text.
#         stop = ['###'], #specifies a sequence of tokens that the GPT model should stop generating text when it encounters
        n = 1, #the number of possible chat completions or responses that the GPT model should generate in response to a given prompt
        messages=[
          {"role":"user", "content": content},
          ])

    # Displaying the output can be helpful if things go wrong
    if print_output:
        print(completions)

    # Return the first choice's text
    ##return completions.choices[0]['message']['content'] #I only want the first repsonses
    return completions.choices[0].message.content.strip()


In [5]:
def parse_output(response:str):
    outputDict = dict()
    if bool(re.match("yes", response, re.I)):
        outputDict["include"] = True
    else:
        outputDict["include"] = False
    outputDict["rationale"] = response
    return outputDict

In [6]:
#Formula to compute the sensitivity of proposed concept set
def compute_sens_and_spec(goldStandardPositives:list, goldStandardNegatives:list, proposedPositives:list, proposedNegatives:list)->float:
    #Convert string elements to numeric elements
    proposedPositives  = list(map(int, proposedPositives))
    proposedNegatives  = list(map(int, proposedNegatives))
    #Initialize variables
    truePositivesArr = sorted(set(proposedPositives).intersection(goldStandardPositives))
    truePositiveCount = len(truePositivesArr)
    falseNegativesArr = sorted(set(proposedNegatives).intersection(goldStandardPositives))
    falseNegativeCount = len(falseNegativesArr)
    trueNegativesArr = sorted(set(proposedNegatives).intersection(goldStandardNegatives))
    trueNegativeCount = len(trueNegativesArr)
    falsePositivesArr = sorted(set(proposedPositives).intersection(goldStandardNegatives))
    falsePositiveCount = len(falsePositivesArr)
    #Initialize dictionary to return      
    returnValuesDict = dict()
    #Avoid division by 0 exception
    if (truePositiveCount+falseNegativeCount) == 0:
        returnValuesDict["sensitivity"] = 0
    else:
        returnValuesDict["sensitivity"] = truePositiveCount/(truePositiveCount + falseNegativeCount)
    returnValuesDict["truePositives"] = truePositivesArr
    returnValuesDict["falseNegatives"] = falseNegativesArr
    if (trueNegativeCount + falsePositiveCount) == 0:
        returnValuesDict["specificity"] = 0
    else:
        returnValuesDict["specificity"] = trueNegativeCount/(trueNegativeCount + falsePositiveCount)
    returnValuesDict["trueNegatives"] = trueNegativesArr
    returnValuesDict["falsePositives"] = falsePositivesArr
    return returnValuesDict

## Type 1 Diabetes

Initialize filepath variables to avoid hardcoding, variables related to the GPT prompt, and any DataFrames needed for storing results

In [19]:
#Define filepath variables
phoebePassOneFilepath = "input/Type1Diabetes/phoebe_concepts_pass_one.csv"
phoebePassTwoFilepath = "input/Type1Diabetes/phoebe_concepts_pass_two.csv"
gptAnswersPassOneFilepath = "output/Type1Diabetes/gpt_answers_pass_one.csv"
gptPositivesPassOneFilepath = "output/Type1Diabetes/gpt_positives_pass_one.txt"
gptNegativesPassOneFilepath = "output/Type1Diabetes/gpt_negatives_pass_one.txt"
gptAnswersPassTwoFilepath = "output/Type1Diabetes/gpt_answers_pass_two.csv"
gptPositivesPassTwoFilepath = "output/Type1Diabetes/gpt_positives_pass_two.txt"
gptNegativesPassTwoFilepath = "output/Type1Diabetes/gpt_negatives_pass_two.txt"
gptFinalFilepath = "output/Type1Diabetes/gpt_concepts_finals.csv"
goldStandardPositivesFilepath = "input/Type1Diabetes/gold_standard_positive_concepts.csv"
goldStandardNegativesFilepath = "input/Type1Diabetes/gold_standard_negative_concepts.csv"
#Define ChatGPT prompt information 
disease = "Type 1 Diabetes Mellitus"
diseaseDescription = "Type 1 Diabetes Mellitus (T1DM) is a chronic autoimmune disorder characterized by the destruction of insulin-producing beta cells in the pancreas, leading to absolute insulin deficiency. It is defined as a metabolic disorder characterized by hyperglycemia due to insulin deficiency resulting from autoimmune destruction of pancreatic beta cells. T1DM typically presents with polyuria, polydipsia, polyphagia, weight loss, and fatigue. Diagnosis is confirmed by elevated blood glucose levels, presence of autoantibodies against pancreatic beta cells, and often necessitates lifelong insulin replacement therapy. Management involves a multidisciplinary approach focusing on insulin therapy, dietary modifications, regular exercise, and monitoring blood glucose levels. Prognosis varies, with complications including diabetic ketoacidosis (DKA), cardiovascular disease, neuropathy, nephropathy, and retinopathy. Exclusions for T1DM include other types of diabetes such as type 2 diabetes mellitus and secondary causes of hyperglycemia (pregnancy, disorders of pancreas, alcohol dependency, etc.)."
domain = "condition"
conditionDefinition = "records of events of a person suggesting the presence of a disease or medical condition stated as a diagnosis, a sign, or a symptom, which is either observed by a provider or reported by the patient"
#Initialize dataframes
dfResponses = pd.DataFrame(columns = ["concept_name", "concept_id", "include", "rationale"])
dfConcepts = pd.read_csv(phoebePassOneFilepath)
#Load gold-standard concept sets
dfGoldStandardPositives = pd.read_csv(goldStandardPositivesFilepath)
goldStandardPositives = dfGoldStandardPositives["concept_id"].tolist()
dfGoldStandardNegatives = pd.read_csv(goldStandardNegativesFilepath)
goldStandardNegatives = dfGoldStandardNegatives["concept_id"].tolist()

### ChatGPT Pass One

In [8]:
for index, row in dfConcepts.iterrows():
    prompt = generate_prompt(conceptName = row["Name"], disease = disease, diseaseDescription = diseaseDescription, domain = "condition", domainDefinition = conditionDefinition)
    res = generate_gpt_response(prompt)
    answerComponents = parse_output(res)
    newRow = {"concept_name": [row["Name"]], "concept_id": [row["Id"]], "include": [answerComponents["include"]], "rationale": [answerComponents["rationale"]]}
    dfNew = pd.DataFrame(newRow)
    dfResponses = pd.concat([dfResponses, dfNew], ignore_index = True)
dfResponses = dfResponses.drop_duplicates()
dfResponses.to_csv(gptAnswersPassOneFilepath, index = False)

In [9]:
gptPositivesMask = dfResponses["include"] == True 
gptNegativesMask = dfResponses["include"] == False
dfPositives = dfResponses[gptPositivesMask]
dfNegatives = dfResponses[gptNegativesMask]
positiveConcepts = dfPositives["concept_id"].tolist()
negativeConcepts = dfNegatives["concept_id"].tolist()
positiveConceptsAsStr = ""
negativeConceptsAsStr = ""
for el in positiveConcepts:
    positiveConceptsAsStr += (str(el) + ",")
for el in negativeConcepts:
    negativeConceptsAsStr += (str(el) + ",")
fp = open(gptPositivesPassOneFilepath, "x")
fp.write(positiveConceptsAsStr[:len(positiveConceptsAsStr)-1])
fp.close()
fp = open(gptNegativesPassOneFilepath, "x")
fp.write(negativeConceptsAsStr[:len(negativeConceptsAsStr)-1])
fp.close()

### ChatGPT Pass Two

In [20]:
dfResponses = pd.DataFrame(columns = ["concept_name", "concept_id", "include", "rationale"])
dfConcepts = pd.read_csv(phoebePassTwoFilepath)
for index, row in dfConcepts.iterrows():
    prompt = generate_prompt(conceptName = row["Name"], disease = disease, diseaseDescription = diseaseDescription, domain = "condition", domainDefinition = conditionDefinition)
    res = generate_gpt_response(prompt)
    answerComponents = parse_output(res)
    newRow = {"concept_name": [row["Name"]], "concept_id": [row["Id"]], "include": [answerComponents["include"]], "rationale": [answerComponents["rationale"]]}
    dfNew = pd.DataFrame(newRow)
    dfResponses = pd.concat([dfResponses, dfNew], ignore_index = True)
dfResponses = dfResponses.drop_duplicates()
dfResponses.to_csv(gptAnswersPassTwoFilepath, index = False)

In [21]:
gptPositivesMask = dfResponses["include"] == True 
gptNegativesMask = dfResponses["include"] == False
dfPositives = dfResponses[gptPositivesMask]
dfNegatives = dfResponses[gptNegativesMask]
positiveConcepts = dfPositives["concept_id"].tolist()
negativeConcepts = dfNegatives["concept_id"].tolist()
positiveConceptsAsStr = ""
negativeConceptsAsStr = ""
for el in positiveConcepts:
    positiveConceptsAsStr += (str(el) + ",")
for el in negativeConcepts:
    negativeConceptsAsStr += (str(el) + ",")
fp = open(gptPositivesPassTwoFilepath, "x")
fp.write(positiveConceptsAsStr[:len(positiveConceptsAsStr)-1])
fp.close()
fp = open(gptNegativesPassTwoFilepath, "x")
fp.write(negativeConceptsAsStr[:len(negativeConceptsAsStr)-1])
fp.close()

In [22]:
#Combine the GPT-recommended concepts from pass one and pass two
fpOne = open(gptPositivesPassOneFilepath, "r")
fpTwo = open(gptPositivesPassTwoFilepath, "r")
positivesAsStr = fpOne.readline() + "," + fpTwo.readline()
fpOne.close()
fpTwo.close()
proposedPositiveConcepts = positivesAsStr.split(",")
#Get rid of empty values from list
proposedPositiveConcepts = list(filter(None, proposedPositiveConcepts))
#Get only unique concept IDs
proposedPositiveConcepts = list(set(proposedPositiveConcepts))
#Convert concept IDs to integers
proposedPositiveConcepts  = list(map(int, proposedPositiveConcepts))
fpOne = open(gptNegativesPassOneFilepath, "r")
fpTwo = open(gptNegativesPassTwoFilepath, "r")
negativesAsStr = fpOne.readline() + "," + fpTwo.readline()
fpOne.close()
fpTwo.close()
proposedNegativeConcepts = negativesAsStr.split(",")
#Get rid of empty values from list
proposedNegativeConcepts = list(filter(None, proposedNegativeConcepts))
#Get only unique concept IDs
proposedNegativeConcepts = list(set(proposedNegativeConcepts))
#Convert concept IDs to integers
proposedNegativeConcepts  = list(map(int, proposedNegativeConcepts))
#Compute sensitivity and specificity
t1dmRes = compute_sens_and_spec(goldStandardPositives=goldStandardPositives, goldStandardNegatives=goldStandardNegatives, proposedPositives=proposedPositiveConcepts, proposedNegatives=proposedNegativeConcepts)
print("Sensitivity: ", t1dmRes["sensitivity"])
print("Specificity: ", t1dmRes["specificity"])

Sensitivity:  0.9529411764705882
Specificity:  0.6842105263157895


## Acute Myocardial Infarction

In [23]:
#Define filepath variables
phoebePassOneFilepath = "input/AcuteMyocardialInfarction/phoebe_concepts_pass_one.csv"
phoebePassTwoFilepath = "input/AcuteMyocardialInfarction/phoebe_concepts_pass_two.csv"
gptAnswersPassOneFilepath = "output/AcuteMyocardialInfarction/gpt_answers_pass_one.csv"
gptPositivesPassOneFilepath = "output/AcuteMyocardialInfarction/gpt_positives_pass_one.txt"
gptNegativesPassOneFilepath = "output/AcuteMyocardialInfarction/gpt_negatives_pass_one.txt"
gptAnswersPassTwoFilepath = "output/AcuteMyocardialInfarction/gpt_answers_pass_two.csv"
gptPositivesPassTwoFilepath = "output/AcuteMyocardialInfarction/gpt_positives_pass_two.txt"
gptNegativesPassTwoFilepath = "output/AcuteMyocardialInfarction/gpt_negatives_pass_two.txt"
gptFinalFilepath = "output/AcuteMyocardialInfarction/gpt_concepts_finals.csv"
goldStandardPositivesFilepath = "input/AcuteMyocardialInfarction/gold_standard_positive_concepts.csv"
goldStandardNegativesFilepath = "input/AcuteMyocardialInfarction/gold_standard_negative_concepts.csv"
#Define ChatGPT prompt information 
disease = "Acute Myocardial Infarction"
diseaseDescription = "Acute myocardial infarction (AMI), commonly known as a heart attack, is a life-threatening medical emergency characterized by the sudden occlusion of a coronary artery, resulting in ischemia and necrosis of cardiac tissue. It is defined as the abrupt interruption of blood flow to a portion of the myocardium, leading to myocardial cell death and subsequent release of cardiac biomarkers such as troponin. AMI typically presents with severe chest pain or pressure, often radiating to the left arm, jaw, or back, along with accompanying symptoms such as shortness of breath, diaphoresis, nausea, and vomiting. Diagnosis is confirmed by clinical history, electrocardiography (ECG) findings indicative of ST-segment elevation or new-onset Q waves, and elevated cardiac biomarkers. Treatment involves immediate reperfusion therapy to restore blood flow to the ischemic myocardium, utilizing thrombolytics or percutaneous coronary intervention (PCI). Additional therapies include antiplatelet agents, anticoagulants, beta-blockers, angiotensin-converting enzyme (ACE) inhibitors, and statins to prevent recurrent ischemic events and reduce mortality. Prognosis varies depending on the extent of myocardial damage, timely intervention, and the presence of comorbidities. Exclusions for AMI include other causes of acute chest pain such as unstable angina, aortic dissection, and pulmonary embolism."
domain = "condition"
conditionDefinition = "records of events of a person suggesting the presence of a disease or medical condition stated as a diagnosis, a sign, or a symptom, which is either observed by a provider or reported by the patient"
#Initialize dataframes
dfResponses = pd.DataFrame(columns = ["concept_name", "concept_id", "include", "rationale"])
dfConcepts = pd.read_csv(phoebePassOneFilepath)
#Load gold-standard concept sets
dfGoldStandardPositives = pd.read_csv(goldStandardPositivesFilepath)
goldStandardPositives = dfGoldStandardPositives["concept_id"].tolist()
dfGoldStandardNegatives = pd.read_csv(goldStandardNegativesFilepath)
goldStandardNegatives = dfGoldStandardNegatives["concept_id"].tolist()

### ChatGPT Pass One

In [11]:
for index, row in dfConcepts.iterrows():
    prompt = generate_prompt(conceptName = row["Name"], disease = disease, diseaseDescription = diseaseDescription, domain = "condition", domainDefinition = conditionDefinition)
    res = generate_gpt_response(prompt)
    answerComponents = parse_output(res)
    newRow = {"concept_name": [row["Name"]], "concept_id": [row["Id"]], "include": [answerComponents["include"]], "rationale": [answerComponents["rationale"]]}
    dfNew = pd.DataFrame(newRow)
    dfResponses = pd.concat([dfResponses, dfNew], ignore_index = True)
dfResponses = dfResponses.drop_duplicates()
dfResponses.to_csv(gptAnswersPassOneFilepath, index = False)

In [12]:
gptPositivesMask = dfResponses["include"] == True 
gptNegativesMask = dfResponses["include"] == False
dfPositives = dfResponses[gptPositivesMask]
dfNegatives = dfResponses[gptNegativesMask]
positiveConcepts = dfPositives["concept_id"].tolist()
negativeConcepts = dfNegatives["concept_id"].tolist()
positiveConceptsAsStr = ""
negativeConceptsAsStr = ""
for el in positiveConcepts:
    positiveConceptsAsStr += (str(el) + ",")
for el in negativeConcepts:
    negativeConceptsAsStr += (str(el) + ",")
fp = open(gptPositivesPassOneFilepath, "x")
fp.write(positiveConceptsAsStr[:len(positiveConceptsAsStr)-1])
fp.close()
fp = open(gptNegativesPassOneFilepath, "x")
fp.write(negativeConceptsAsStr[:len(negativeConceptsAsStr)-1])
fp.close()

### ChatGPT Pass Two

In [24]:
dfResponses = pd.DataFrame(columns = ["concept_name", "concept_id", "include", "rationale"])
dfConcepts = pd.read_csv(phoebePassTwoFilepath)
for index, row in dfConcepts.iterrows():
    prompt = generate_prompt(conceptName = row["Name"], disease = disease, diseaseDescription = diseaseDescription, domain = "condition", domainDefinition = conditionDefinition)
    res = generate_gpt_response(prompt)
    answerComponents = parse_output(res)
    newRow = {"concept_name": [row["Name"]], "concept_id": [row["Id"]], "include": [answerComponents["include"]], "rationale": [answerComponents["rationale"]]}
    dfNew = pd.DataFrame(newRow)
    dfResponses = pd.concat([dfResponses, dfNew], ignore_index = True)
dfResponses = dfResponses.drop_duplicates()
dfResponses.to_csv(gptAnswersPassTwoFilepath, index = False)

In [25]:
gptPositivesMask = dfResponses["include"] == True 
gptNegativesMask = dfResponses["include"] == False
dfPositives = dfResponses[gptPositivesMask]
dfNegatives = dfResponses[gptNegativesMask]
positiveConcepts = dfPositives["concept_id"].tolist()
negativeConcepts = dfNegatives["concept_id"].tolist()
positiveConceptsAsStr = ""
negativeConceptsAsStr = ""
for el in positiveConcepts:
    positiveConceptsAsStr += (str(el) + ",")
for el in negativeConcepts:
    negativeConceptsAsStr += (str(el) + ",")
fp = open(gptPositivesPassTwoFilepath, "x")
fp.write(positiveConceptsAsStr[:len(positiveConceptsAsStr)-1])
fp.close()
fp = open(gptNegativesPassTwoFilepath, "x")
fp.write(negativeConceptsAsStr[:len(negativeConceptsAsStr)-1])
fp.close()

In [26]:
#Combine the GPT-recommended concepts from pass one and pass two
fpOne = open(gptPositivesPassOneFilepath, "r")
fpTwo = open(gptPositivesPassTwoFilepath, "r")
positivesAsStr = fpOne.readline() + "," + fpTwo.readline()
fpOne.close()
fpTwo.close()
proposedPositiveConcepts = positivesAsStr.split(",")
#Get rid of empty values from list
proposedPositiveConcepts = list(filter(None, proposedPositiveConcepts))
#Get only unique concept IDs
proposedPositiveConcepts = list(set(proposedPositiveConcepts))
#Convert concept IDs to integers
proposedPositiveConcepts  = list(map(int, proposedPositiveConcepts))
fpOne = open(gptNegativesPassOneFilepath, "r")
fpTwo = open(gptNegativesPassTwoFilepath, "r")
negativesAsStr = fpOne.readline() + "," + fpTwo.readline()
fpOne.close()
fpTwo.close()
proposedNegativeConcepts = negativesAsStr.split(",")
#Get rid of empty values from list
proposedNegativeConcepts = list(filter(None, proposedNegativeConcepts))
#Get only unique concept IDs
proposedNegativeConcepts = list(set(proposedNegativeConcepts))
#Convert concept IDs to integers
proposedNegativeConcepts  = list(map(int, proposedNegativeConcepts))
#Compute sensitivity and specificity
amiRes = compute_sens_and_spec(goldStandardPositives=goldStandardPositives, goldStandardNegatives=goldStandardNegatives, proposedPositives=proposedPositiveConcepts, proposedNegatives=proposedNegativeConcepts)
print("Sensitivity: ", amiRes["sensitivity"])
print("Specificity: ", amiRes["specificity"])

Sensitivity:  0.8787878787878788
Specificity:  0.4794520547945205


## Rheumatoid Arthritis

In [27]:
#Define filepath variables
phoebePassOneFilepath = "input/RheumatoidArthritis/phoebe_concepts_pass_one.csv"
phoebePassTwoFilepath = "input/RheumatoidArthritis/phoebe_concepts_pass_two.csv"
gptAnswersPassOneFilepath = "output/RheumatoidArthritis/gpt_answers_pass_one.csv"
gptPositivesPassOneFilepath = "output/RheumatoidArthritis/gpt_positives_pass_one.txt"
gptNegativesPassOneFilepath = "output/RheumatoidArthritis/gpt_negatives_pass_one.txt"
gptAnswersPassTwoFilepath = "output/RheumatoidArthritis/gpt_answers_pass_two.csv"
gptPositivesPassTwoFilepath = "output/RheumatoidArthritis/gpt_positives_pass_two.txt"
gptNegativesPassTwoFilepath = "output/RheumatoidArthritis/gpt_negatives_pass_two.txt"
gptFinalFilepath = "output/RheumatoidArthritis/gpt_concepts_finals.csv"
goldStandardPositivesFilepath = "input/RheumatoidArthritis/gold_standard_positive_concepts.csv"
goldStandardNegativesFilepath = "input/RheumatoidArthritis/gold_standard_negative_concepts.csv"
#Define ChatGPT prompt information 
disease = "Rheumatoid Arthritis"
diseaseDescription = "Rheumatoid Arthritis (RA) is a chronic autoimmune inflammatory disorder primarily affecting the synovial joints. It is defined as a systemic autoimmune disease characterized by symmetric polyarthritis with joint swelling, tenderness, and destruction, leading to deformities and functional impairment. RA often presents with morning stiffness, joint pain, and swelling, particularly in the small joints of the hands and feet. Diagnosis is based on clinical criteria such as joint involvement, serological markers (e.g., rheumatoid factor, anti-cyclic citrullinated peptide antibodies), and imaging findings (e.g., joint erosions on X-ray). Treatment aims to control inflammation, relieve symptoms, and prevent joint damage, utilizing disease-modifying antirheumatic drugs (DMARDs), biologic agents, nonsteroidal anti-inflammatory drugs (NSAIDs), and glucocorticoids. Prognosis varies widely, with some patients achieving remission while others experience progressive joint damage and disability. Exclusions for RA include other forms of arthritis such as osteoarthritis and systemic lupus erythematosus (SLE)."
domain = "condition"
conditionDefinition = "records of events of a person suggesting the presence of a disease or medical condition stated as a diagnosis, a sign, or a symptom, which is either observed by a provider or reported by the patient"
#Initialize dataframes
dfResponses = pd.DataFrame(columns = ["concept_name", "concept_id", "include", "rationale"])
dfConcepts = pd.read_csv(phoebePassOneFilepath)
#Load gold-standard concept sets
dfGoldStandardPositives = pd.read_csv(goldStandardPositivesFilepath)
goldStandardPositives = dfGoldStandardPositives["concept_id"].tolist()
dfGoldStandardNegatives = pd.read_csv(goldStandardNegativesFilepath)
goldStandardNegatives = dfGoldStandardNegatives["concept_id"].tolist()

### ChatGPT Pass One

In [14]:
for index, row in dfConcepts.iterrows():
    prompt = generate_prompt(conceptName = row["Name"], disease = disease, diseaseDescription = diseaseDescription, domain = "condition", domainDefinition = conditionDefinition)
    res = generate_gpt_response(prompt)
    answerComponents = parse_output(res)
    newRow = {"concept_name": [row["Name"]], "concept_id": [row["Id"]], "include": [answerComponents["include"]], "rationale": [answerComponents["rationale"]]}
    dfNew = pd.DataFrame(newRow)
    dfResponses = pd.concat([dfResponses, dfNew], ignore_index = True)
dfResponses = dfResponses.drop_duplicates()
dfResponses.to_csv(gptAnswersPassOneFilepath, index = False)

In [15]:
gptPositivesMask = dfResponses["include"] == True 
gptNegativesMask = dfResponses["include"] == False
dfPositives = dfResponses[gptPositivesMask]
dfNegatives = dfResponses[gptNegativesMask]
positiveConcepts = dfPositives["concept_id"].tolist()
negativeConcepts = dfNegatives["concept_id"].tolist()
positiveConceptsAsStr = ""
negativeConceptsAsStr = ""
for el in positiveConcepts:
    positiveConceptsAsStr += (str(el) + ",")
for el in negativeConcepts:
    negativeConceptsAsStr += (str(el) + ",")
fp = open(gptPositivesPassOneFilepath, "x")
fp.write(positiveConceptsAsStr[:len(positiveConceptsAsStr)-1])
fp.close()
fp = open(gptNegativesPassOneFilepath, "x")
fp.write(negativeConceptsAsStr[:len(negativeConceptsAsStr)-1])
fp.close()

### ChatGPT Pass Two

In [28]:
dfResponses = pd.DataFrame(columns = ["concept_name", "concept_id", "include", "rationale"])
dfConcepts = pd.read_csv(phoebePassTwoFilepath)
for index, row in dfConcepts.iterrows():
    prompt = generate_prompt(conceptName = row["Name"], disease = disease, diseaseDescription = diseaseDescription, domain = "condition", domainDefinition = conditionDefinition)
    res = generate_gpt_response(prompt)
    answerComponents = parse_output(res)
    newRow = {"concept_name": [row["Name"]], "concept_id": [row["Id"]], "include": [answerComponents["include"]], "rationale": [answerComponents["rationale"]]}
    dfNew = pd.DataFrame(newRow)
    dfResponses = pd.concat([dfResponses, dfNew], ignore_index = True)
dfResponses = dfResponses.drop_duplicates()
dfResponses.to_csv(gptAnswersPassTwoFilepath, index = False)

In [29]:
gptPositivesMask = dfResponses["include"] == True 
gptNegativesMask = dfResponses["include"] == False
dfPositives = dfResponses[gptPositivesMask]
dfNegatives = dfResponses[gptNegativesMask]
positiveConcepts = dfPositives["concept_id"].tolist()
negativeConcepts = dfNegatives["concept_id"].tolist()
positiveConceptsAsStr = ""
negativeConceptsAsStr = ""
for el in positiveConcepts:
    positiveConceptsAsStr += (str(el) + ",")
for el in negativeConcepts:
    negativeConceptsAsStr += (str(el) + ",")
fp = open(gptPositivesPassTwoFilepath, "x")
fp.write(positiveConceptsAsStr[:len(positiveConceptsAsStr)-1])
fp.close()
fp = open(gptNegativesPassTwoFilepath, "x")
fp.write(negativeConceptsAsStr[:len(negativeConceptsAsStr)-1])
fp.close()

In [30]:
#Combine the GPT-recommended concepts from pass one and pass two
fpOne = open(gptPositivesPassOneFilepath, "r")
fpTwo = open(gptPositivesPassTwoFilepath, "r")
positivesAsStr = fpOne.readline() + "," + fpTwo.readline()
fpOne.close()
fpTwo.close()
proposedPositiveConcepts = positivesAsStr.split(",")
#Get rid of empty values from list
proposedPositiveConcepts = list(filter(None, proposedPositiveConcepts))
#Get only unique concept IDs
proposedPositiveConcepts = list(set(proposedPositiveConcepts))
#Convert concept IDs to integers
proposedPositiveConcepts  = list(map(int, proposedPositiveConcepts))
fpOne = open(gptNegativesPassOneFilepath, "r")
fpTwo = open(gptNegativesPassTwoFilepath, "r")
negativesAsStr = fpOne.readline() + "," + fpTwo.readline()
fpOne.close()
fpTwo.close()
proposedNegativeConcepts = negativesAsStr.split(",")
#Get rid of empty values from list
proposedNegativeConcepts = list(filter(None, proposedNegativeConcepts))
#Get only unique concept IDs
proposedNegativeConcepts = list(set(proposedNegativeConcepts))
#Convert concept IDs to integers
proposedNegativeConcepts  = list(map(int, proposedNegativeConcepts))
#Compute sensitivity and specificity
raRes = compute_sens_and_spec(goldStandardPositives=goldStandardPositives, goldStandardNegatives=goldStandardNegatives, proposedPositives=proposedPositiveConcepts, proposedNegatives=proposedNegativeConcepts)
print("Sensitivity: ", raRes["sensitivity"])
print("Specificity: ", raRes["specificity"])

Sensitivity:  0.9411764705882353
Specificity:  0.4298245614035088


## Pulmonary Hypertension

In [32]:
#Define filepath variables
phoebePassOneFilepath = "input/PulmonaryHypertension/phoebe_concepts_pass_one.csv"
phoebePassTwoFilepath = "input/PulmonaryHypertension/phoebe_concepts_pass_two.csv"
gptAnswersPassOneFilepath = "output/PulmonaryHypertension/gpt_answers_pass_one.csv"
gptPositivesPassOneFilepath = "output/PulmonaryHypertension/gpt_positives_pass_one.txt"
gptNegativesPassOneFilepath = "output/PulmonaryHypertension/gpt_negatives_pass_one.txt"
gptAnswersPassTwoFilepath = "output/PulmonaryHypertension/gpt_answers_pass_two.csv"
gptPositivesPassTwoFilepath = "output/PulmonaryHypertension/gpt_positives_pass_two.txt"
gptNegativesPassTwoFilepath = "output/PulmonaryHypertension/gpt_negatives_pass_two.txt"
gptFinalFilepath = "output/PulmonaryHypertension/gpt_concepts_finals.csv"
goldStandardPositivesFilepath = "input/PulmonaryHypertension/gold_standard_positive_concepts.csv"
goldStandardNegativesFilepath = "input/PulmonaryHypertension/gold_standard_negative_concepts.csv"
#Define ChatGPT prompt information 
disease = "Pulmonary Hypertension"
diseaseDescription = "Pulmonary Hypertension (PH) is a complex and progressive condition characterized by elevated blood pressure within the pulmonary vasculature, leading to right ventricular dysfunction and ultimately, heart failure. It is defined as a hemodynamic and pathophysiological state characterized by an increase in mean pulmonary arterial pressure (mPAP) greater than or equal to 20 mmHg at rest, as assessed by right heart catheterization. PH can result from various etiologies, including idiopathic, heritable, drug-induced, and associated with other conditions such as connective tissue diseases, congenital heart defects, or chronic lung diseases. Patients with PH may present with symptoms such as dyspnea, fatigue, chest pain, syncope, and signs of right heart failure. Diagnostic evaluation involves a thorough clinical assessment, echocardiography, pulmonary function tests, and right heart catheterization to confirm the diagnosis and assess disease severity. Treatment aims to improve symptoms, slow disease progression, and optimize hemodynamics, utilizing various classes of medications including vasodilators, endothelin receptor antagonists, phosphodiesterase-5 inhibitors, soluble guanylate cyclase stimulators, and prostacyclin analogs. Prognosis varies depending on the underlying cause and severity of PH, with early diagnosis and targeted therapy improving outcomes. Exclusions for PH include other causes of pulmonary arterial hypertension such as left heart disease, chronic thromboembolic disease, and pulmonary arterial hypertension associated with lung diseases and/or hypoxia."
domain = "condition"
conditionDefinition = "records of events of a person suggesting the presence of a disease or medical condition stated as a diagnosis, a sign, or a symptom, which is either observed by a provider or reported by the patient"
#Initialize dataframes
dfResponses = pd.DataFrame(columns = ["concept_name", "concept_id", "include", "rationale"])
dfConcepts = pd.read_csv(phoebePassOneFilepath)
#Load gold-standard concept sets
dfGoldStandardPositives = pd.read_csv(goldStandardPositivesFilepath)
goldStandardPositives = dfGoldStandardPositives["concept_id"].tolist()
dfGoldStandardNegatives = pd.read_csv(goldStandardNegativesFilepath)
goldStandardNegatives = dfGoldStandardNegatives["concept_id"].tolist()

### ChatGPT Pass One

In [17]:
for index, row in dfConcepts.iterrows():
    prompt = generate_prompt(conceptName = row["Name"], disease = disease, diseaseDescription = diseaseDescription, domain = "condition", domainDefinition = conditionDefinition)
    res = generate_gpt_response(prompt)
    answerComponents = parse_output(res)
    newRow = {"concept_name": [row["Name"]], "concept_id": [row["Id"]], "include": [answerComponents["include"]], "rationale": [answerComponents["rationale"]]}
    dfNew = pd.DataFrame(newRow)
    dfResponses = pd.concat([dfResponses, dfNew], ignore_index = True)
dfResponses = dfResponses.drop_duplicates()
dfResponses.to_csv(gptAnswersPassOneFilepath, index = False)

In [18]:
gptPositivesMask = dfResponses["include"] == True 
gptNegativesMask = dfResponses["include"] == False
dfPositives = dfResponses[gptPositivesMask]
dfNegatives = dfResponses[gptNegativesMask]
positiveConcepts = dfPositives["concept_id"].tolist()
negativeConcepts = dfNegatives["concept_id"].tolist()
positiveConceptsAsStr = ""
negativeConceptsAsStr = ""
for el in positiveConcepts:
    positiveConceptsAsStr += (str(el) + ",")
for el in negativeConcepts:
    negativeConceptsAsStr += (str(el) + ",")
fp = open(gptPositivesPassOneFilepath, "x")
fp.write(positiveConceptsAsStr[:len(positiveConceptsAsStr)-1])
fp.close()
fp = open(gptNegativesPassOneFilepath, "x")
fp.write(negativeConceptsAsStr[:len(negativeConceptsAsStr)-1])
fp.close()

### ChatGPT Pass Two

In [34]:
dfResponses = pd.DataFrame(columns = ["concept_name", "concept_id", "include", "rationale"])
dfConcepts = pd.read_csv(phoebePassTwoFilepath)
for index, row in dfConcepts.iterrows():
    prompt = generate_prompt(conceptName = row["Name"], disease = disease, diseaseDescription = diseaseDescription, domain = "condition", domainDefinition = conditionDefinition)
    res = generate_gpt_response(prompt)
    answerComponents = parse_output(res)
    newRow = {"concept_name": [row["Name"]], "concept_id": [row["Id"]], "include": [answerComponents["include"]], "rationale": [answerComponents["rationale"]]}
    dfNew = pd.DataFrame(newRow)
    dfResponses = pd.concat([dfResponses, dfNew], ignore_index = True)
dfResponses = dfResponses.drop_duplicates()
dfResponses.to_csv(gptAnswersPassTwoFilepath, index = False)

In [35]:
gptPositivesMask = dfResponses["include"] == True 
gptNegativesMask = dfResponses["include"] == False
dfPositives = dfResponses[gptPositivesMask]
dfNegatives = dfResponses[gptNegativesMask]
positiveConcepts = dfPositives["concept_id"].tolist()
negativeConcepts = dfNegatives["concept_id"].tolist()
positiveConceptsAsStr = ""
negativeConceptsAsStr = ""
for el in positiveConcepts:
    positiveConceptsAsStr += (str(el) + ",")
for el in negativeConcepts:
    negativeConceptsAsStr += (str(el) + ",")
fp = open(gptPositivesPassTwoFilepath, "x")
fp.write(positiveConceptsAsStr[:len(positiveConceptsAsStr)-1])
fp.close()
fp = open(gptNegativesPassTwoFilepath, "x")
fp.write(negativeConceptsAsStr[:len(negativeConceptsAsStr)-1])
fp.close()

In [36]:
#Combine the GPT-recommended concepts from pass one and pass two
fpOne = open(gptPositivesPassOneFilepath, "r")
fpTwo = open(gptPositivesPassTwoFilepath, "r")
positivesAsStr = fpOne.readline() + "," + fpTwo.readline()
fpOne.close()
fpTwo.close()
proposedPositiveConcepts = positivesAsStr.split(",")
#Get rid of empty values from list
proposedPositiveConcepts = list(filter(None, proposedPositiveConcepts))
#Get only unique concept IDs
proposedPositiveConcepts = list(set(proposedPositiveConcepts))
#Convert concept IDs to integers
proposedPositiveConcepts  = list(map(int, proposedPositiveConcepts))
fpOne = open(gptNegativesPassOneFilepath, "r")
fpTwo = open(gptNegativesPassTwoFilepath, "r")
negativesAsStr = fpOne.readline() + "," + fpTwo.readline()
fpOne.close()
fpTwo.close()
proposedNegativeConcepts = negativesAsStr.split(",")
#Get rid of empty values from list
proposedNegativeConcepts = list(filter(None, proposedNegativeConcepts))
#Get only unique concept IDs
proposedNegativeConcepts = list(set(proposedNegativeConcepts))
#Convert concept IDs to integers
proposedNegativeConcepts  = list(map(int, proposedNegativeConcepts))
#Compute sensitivity and specificity
phRes = compute_sens_and_spec(goldStandardPositives=goldStandardPositives, goldStandardNegatives=goldStandardNegatives, proposedPositives=proposedPositiveConcepts, proposedNegatives=proposedNegativeConcepts)
print("Sensitivity: ", phRes["sensitivity"])
print("Specificity: ", phRes["specificity"])

Sensitivity:  0
Specificity:  0.6933333333333334
